In [1]:
from openpyxl import load_workbook
import os
import numpy as np
from numpy.random import randn
import pandas as pd
from pandas import Series, DataFrame
from dateutil.parser import parse
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import csv
import datetime
import time
from datetime import datetime, timedelta

In [2]:
df_list = [
    #df1 = pd.read_csv('202107_TFM.csv')
    #df2 = pd.read_csv('202108_TFM.csv')
    #df3 = pd.read_csv('202109_TFM.csv')
    #pd.read_csv('202110_TFM.csv'),
    pd.read_csv('202111_TFM.csv'),
    pd.read_csv('202112_TFM.csv'),
    pd.read_csv('202201_TFM.csv'),
    pd.read_csv('202202_TFM.csv'),
    pd.read_csv('202203_TFM.csv'),
    pd.read_csv('202204_TFM.csv'),
    pd.read_csv('202205_TFM.csv'),
    pd.read_csv('202206_TFM.csv'),
    pd.read_csv('202207_TFM.csv'),
    #pd.read_csv('202208_TFM.csv'),
    #pd.read_csv('202209_TFM.csv')
    #pd.read_csv('202210_TFM.csv'),
    #pd.read_csv('202211_TFM.csv'),
    #pd.read_csv('202212_TFM.csv'),
    #pd.read_csv('202301_TFM.csv'),
    #pd.read_csv('202302_TFM.csv'),
    #pd.read_csv('202303_TFM.csv'),
    #pd.read_csv('202304_TFM.csv'),
    #pd.read_csv('202305_TFM.csv'),
    #pd.read_csv('202306_TFM.csv'),
    #pd.read_csv('202307_TFM.csv'),
    #pd.read_csv('202308_TFM.csv')
]

In [3]:
df = pd.concat(df_list)

In [4]:
m_list = [
    #pd.read_csv('202110_TFM2.csv', dtype={'Product': str}),
    pd.read_csv('202111_TFM2.csv', dtype={'Product': str}),
    pd.read_csv('202112_TFM2.csv', dtype={'Product': str}),
    pd.read_csv('202201_TFM2.csv', dtype={'Product': str}),
    pd.read_csv('202202_TFM2.csv', dtype={'Product': str}),
    pd.read_csv('202203_TFM2.csv', dtype={'Product': str}),
    pd.read_csv('202204_TFM2.csv', dtype={'Product': str}),
    pd.read_csv('202205~202301TFM.csv'),
    #pd.read_csv('202302_TFM2.csv'),
    #pd.read_csv('202303_TFM2.csv'),
    #pd.read_csv('202304_TFM2.csv'),
    #pd.read_csv('202305_TFM2.csv'),
    #pd.read_csv('202306_TFM2.csv'),
    #pd.read_csv('202307_TFM2.csv'),
    #pd.read_csv('202308_TFM2.csv')
]

In [5]:
pq = pd.read_csv('EQ_Process_BN.csv')

In [6]:
pq['EQ_ID'] = pq['EQ_ID'].str.strip()

In [7]:
m = pd.concat(m_list)

In [8]:
m.rename(columns={'Ident':'Metakey Ident'},inplace=True)

In [9]:
device = pd.read_csv('Device.csv')

In [10]:
df = pd.merge(df, m[['Metakey Ident','Device']], on="Metakey Ident", how='left')

In [11]:
df = pd.merge(df, device, on="Device", how='left')

In [12]:
df = pd.merge(df, m[['Metakey Ident','Lotid']], on="Metakey Ident", how='left')

In [13]:
EQ_ident = pd.read_csv('EQ_ident.csv')

In [14]:
EQ_states = pd.read_csv('EQ_states.csv')

In [15]:
Lv5_Indent = pd.read_csv('Lv5_indent.csv')

In [16]:
Lv3_Indent = pd.read_csv('Lv3_indent.csv')

In [17]:
Lv5_Indent.rename(columns={'STATE_IDENT':'Tr25 5 Status'},inplace=True)

In [18]:
df = pd.merge(df, EQ_ident, on='Eq Ident', how = 'inner')

In [19]:
df['Process'] = df['EQ_ID'].str.extract(pat = '([A-Z]..)')

In [20]:
df = pd.merge(df, Lv3_Indent, on='Tr25 3 Status', how = 'inner')

In [21]:
df = pd.merge(df, Lv5_Indent, on='Tr25 5 Status', how = 'inner')

In [22]:
df = pd.merge(df, EQ_states, on='Eq Status', how = 'outer')

In [23]:
df = df.dropna(subset=['Timestamp End', 'Timestamp Start'])

In [24]:
df['Timestamp Start'] = pd.to_datetime(df['Timestamp Start'], format='%Y-%m-%d %p %I:%M:%S', errors='ignore')

In [25]:
def convert_to_24h_format(dt_str):
    parts = dt_str.split(' ')
    date = parts[0]
    time_parts = parts[2].split(':')
    hours = int(time_parts[0])
    if parts[1] == '오후' and hours != 12:
        hours += 12
    elif parts[1] == '오전' and hours == 12:
        hours = 0
    time = f'{hours:02d}:{time_parts[1]}:{time_parts[2]}'
    return f'{date} {time}'

In [26]:
convert_to_24h_format_v = np.vectorize(convert_to_24h_format)
df['Timestamp Start'] = convert_to_24h_format_v(df['Timestamp Start'].values)
df['Timestamp Start'] = pd.to_datetime(df['Timestamp Start'], format='%Y-%m-%d %H:%M:%S')
#df['Timestamp Start'] = df['Timestamp Start'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [27]:
convert_to_24h_format_v = np.vectorize(convert_to_24h_format)
df['Timestamp End'] = convert_to_24h_format_v(df['Timestamp End'].values)
df['Timestamp End'] = pd.to_datetime(df['Timestamp End'], format='%Y-%m-%d %H:%M:%S')
#df['Timestamp End'] = df['Timestamp End'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [28]:
df['seconds'] = (df['Timestamp End'] - df['Timestamp Start']).dt.total_seconds().astype(int)

In [29]:
#weekly data 뽑을때

df['Timestamp Start'] = pd.to_datetime(df['Timestamp Start'])
start_time = datetime(2022, 10, 1, 6, 0, 0)
df['time_difference'] = df['Timestamp Start'] - start_time
df['week'] = (df['time_difference'] / timedelta(weeks=1)).astype(int) + 1
df.drop('time_difference', axis=1, inplace=True)


In [29]:
#df['Month'] = df['Timestamp Start'].dt.strftime('%Y-%m')

In [30]:
#df['Time'] = df['Timestamp Start'].dt.strftime("%H:%M:%S")
#df['Date'] = pd.to_datetime(df['Timestamp Start']).dt.to_period('D')

In [30]:
df = df[['Timestamp Start', 'week','Process','EQ_ID','Lotid','Product type','STATE_NAME_x', 'STATE_NAME_y','seconds']]

In [31]:
df.sort_values(by=['EQ_ID','Timestamp Start'], inplace=True)

In [32]:
df['Lotid'] = df['Lotid'].fillna(method = 'ffill')
df['Lotid'] = df['Lotid'].fillna(method = 'bfill')

In [33]:
df['Product type'] = df['Product type'].fillna(method = 'ffill')
df['Product type'] = df['Product type'].fillna(method = 'bfill')

In [34]:
df = df[['week', 'Process','EQ_ID', 'Lotid', 'Product type','STATE_NAME_x', 'STATE_NAME_y','seconds',]]

In [36]:
df.head()

,week,Process,EQ_ID,Lotid,Product type,STATE_NAME_x,STATE_NAME_y,seconds
2382448,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Undefined Waiting,Undefined Waiting,45
7355461,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Normal Production,Normal Production,47
7355462,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Normal Production,Normal Production,773
10660250,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Waiting for Response,Waiting for Response,31
10660251,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Waiting for Response,Waiting for Response,78


In [38]:
df['group'] = (df['STATE_NAME_y'] != df['STATE_NAME_y'].shift()).cumsum()
df2 = df.groupby('group').agg({'week': 'first', 'Process': 'first', 'EQ_ID': 'first', 'Lotid': 'first' , 'Product type': 'first', 'STATE_NAME_x': 'first', 'STATE_NAME_y': 'first', 'seconds': 'sum'})
df2 = df2.reset_index(drop=False)

In [39]:
df2 = df2.drop(['group'], axis=1)

In [40]:
df2.head()

,week,Process,EQ_ID,Lotid,Product type,STATE_NAME_x,STATE_NAME_y,seconds
0,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Undefined Waiting,Undefined Waiting,45
1,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Normal Production,Normal Production,820
2,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Waiting for Response,Waiting for Response,109
3,-46,AFV,AFVI001,2Z1420T6,IKCM10H60GA,Normal Production,Normal Production,253
4,-46,AFV,AFVI001,2Z1420HH,IKCM10H60GA,Undefined Waiting,Undefined Waiting,15


In [38]:
#Bottleneck base analysis
PFC_D = ['IM564-X6D','IFCM30T65GD','IFCM20T65GD', 'IFCM15S60GS', 'IFCM15P60GD', 'IFCM15S60GD']
PFC_EQ = ['LSD011', 'LSD017','LSD028', 'LSD029', 'LSD040', 'LSD039']
PFC_EQ2 = ['LSD011', 'LSD027','LSD029', 'LSD038']
#df.sort_values(by=['EQ_ID','Timestamp Start'], inplace=True)
#df['Product type'] = np.where(df['Product type'].fillna(method = 'bfill'), df['Product type'])
#df['Product type'] = np.where(df['Product type'].fillna(method = 'ffill'), df['Product type'])

df2 = df2[~( (df2['Product type'].isin(PFC_D)) & (df2['EQ_ID'].isin(PFC_EQ))) ]
df2 = df2[~( (~df2['Product type'].isin(PFC_D)) & (df2['EQ_ID'].isin(PFC_EQ2))) ]

In [51]:
'''
df2 = df2[(df2.STATE_NAME_x != 'No Material')]
#df = df[(df.LEVEL5_NAME != 'SPEED LOSS')]
df2 = df2[(df2.STATE_NAME_x != 'Ivalid Data')]
df2 = df2[(df2.STATE_NAME_x != 'Exception')]
df2 = df2[(df2.STATE_NAME_x != 'No Communication')]
df2 = df2[(df2.STATE_NAME_x != 'Non-Scheduled Time')]
'''

In [ ]:
df4 = df2.groupby(['Date','Lotid']).count()

In [36]:
#FP SDA
filter1 = df2['EQ_ID'].isin(['LSD003', 'LSD004', 'LSD005', 'LSD006', 'LSD007', 'LSD009', 'LSD010', 'LSD013', 'LSD014', 'LSD015', 'LSD016', 'LSD018', 'LSD019', 'LSD024', 'LSD025', 'LSD026', 'LSD036', 'LSD037'])

In [51]:
filter1 = df2['EQ_ID'].isin(['LSD001', 'LSD003', 'LSD004', 'LSD005', 'LSD006', 'LSD007', 'LSD009', 'LSD010', 'LSD012', 'LSD013', 'LSD014', 'LSD015', 'LSD016', 'LSD018', 'LSD019', 'LSD020', 'LSD021', 'LSD023', 'LSD024', 'LSD025', 'LSD026', 'LSD027', 'LSD030', 'LSD031', 'LSD035', 'LSD036', 'LSD037', 'LSD038', 'LSD041'])

In [25]:
#Al WB all
filter1 = df2['EQ_ID'].isin(['LAW004', 'LAW006', 'LAW007', 'LAW008', 'LAW009', 'LAW010', 'LAW011', 'LAW012', 'LAW013', 'LAW016', 'LAW018', 'LAW020', 'LAW022', 'LAW024', 'LAW026', 'LAW030', 'LAW033', 'LAW034', 'LAW035', 'LAW036', 'LAW040', 'LAW043', 'LAW050', 'LAW028', 'LAW029', 'LAW032', 'LAW038', 'LAW045', 'LAW046', 'LAW053', 'LAW014', 'LAW037', 'LAW042', 'LAW044', 'LAW052', 'LAW041', 'LAW047', 'LAW031', 'LAW039', 'LAW049', 'LAW048', 'LAW051'])

In [41]:
#FP Al WB
filter1 = df2['EQ_ID'].isin(['LAW004','LAW005', 'LAW006', 'LAW007', 'LAW008', 'LAW009', 'LAW010', 'LAW011', 'LAW012', 'LAW013', 'LAW016', 'LAW018', 'LAW020', 'LAW022', 'LAW024', 'LAW026', 'LAW030', 'LAW033', 'LAW034', 'LAW035', 'LAW036', 'LAW040', 'LAW043'])

In [58]:
#DCG P&G
filter1 = df2['EQ_ID'].isin(['LAW029', 'LAW028', 'LAW038', 'LAW032', 'LAW046', 'LAW045', 'LAW053'])

In [78]:
#Tiny1.0 3x
filter1 = df2['EQ_ID'].isin(['LAW031', 'LAW039', 'LAW049'])

In [39]:
#BN SDA
filter1 = df2['EQ_ID'].isin(['LSD003', 'LSD004', 'LSD005', 'LSD006', 'LSD007', 'LSD009', 'LSD010', 'LSD013', 'LSD014', 'LSD015', 'LSD016', 'LSD018', 'LSD019', 'LSD024', 'LSD025', 'LSD026', 'LSD036', 'LSD037','LSD001', 'LSD011', 'LSD012', 'LSD017', 'LSD023', 'LSD027', 'LSD028', 'LSD029', 'LSD035', 'LSD038', 'LSD039', 'LSD040','LSD020', 'LSD021', 'LSD030', 'LSD031','LSD041'])

In [42]:
#Mold
filter1 = df2['EQ_ID'].isin(['LMD002', 'LMD003', 'LMD004', 'LMD005', 'LMD006', 'LMD007', 'LMD009', 'LMD010', 'LMD011', 'LMD012', 'LMD013', 'LMD14'])

In [42]:
#All SDA
filter1 = df2['EQ_ID'].isin(['LSD001', 'LSD002', 'LSD003', 'LSD004', 'LSD005', 'LSD006', 'LSD007', 'LSD008', 'LSD009', 'LSD010', 'LSD011', 'LSD012', 'LSD013', 'LSD014', 'LSD015', 'LSD016', 'LSD017', 'LSD018', 'LSD019', 'LSD020', 'LSD021', 'LSD022', 'LSD023', 'LSD024', 'LSD025', 'LSD026', 'LSD027', 'LSD028', 'LSD029', 'LSD030', 'LSD031', 'LSD035', 'LSD036', 'LSD037', 'LSD038', 'LSD039', 'LSD040', 'LSD041', 'LSD042'])

In [35]:
#PCB connection
filter1 = df2['EQ_ID'].isin(['LPC003','LPC004','LPC005'])

In [42]:
df3 = df2[filter1]

In [43]:
df3.head()

,week,Process,EQ_ID,Lotid,Product type,STATE_NAME_x,STATE_NAME_y,seconds
3799569,-46,LAW,LAW004,2Z2300J3,IM393-S6F,Waiting for Response,Waiting for Response,9
3799570,-46,LAW,LAW004,2Z2300J3,IM393-S6F,Normal Production,Normal Production,604
3799571,-46,LAW,LAW004,2Z2300J3,IM393-S6F,Maintenance,Change Tool,1140
3799572,-46,LAW,LAW004,2Z2300J3,IM393-S6F,Normal Production,Normal Production,1390
3799573,-46,LAW,LAW004,2Z2300J3,IM393-S6F,Waiting for Response,Waiting for Response,11


In [73]:
#Nomaterial 앞에 있는 Waiting for response 빼고 계산법
df3 = df3.reset_index()
k = df3.index[df3['STATE_NAME_x'] == "No Material" ].tolist()
lst=[]
w=[]
for i in k:
    if df3.iloc[i-1]['STATE_NAME_x'] == "Waiting for Response":
        lst.append(i-1)

w = lst + k
df3 = df3[~df3.index.isin(lst)]
#df3 = df3[(df3.STATE_NAME_x != 'No Material')]

In [57]:
df4 = df3[df3.index.isin(lst)]

In [59]:
df5 = df4[(df4.seconds > 0)] 

In [60]:
df5.sort_values(by=['seconds'], inplace=True)

In [63]:
df5['seconds'].mean()

100.08821902654867

In [38]:
df4 = pd.crosstab([df3.STATE_NAME_x],df3.Month,values=df3.seconds,aggfunc=sum,rownames=['Level 5'],colnames=['month'], normalize='columns')

In [51]:
df3.to_excel("Mold 2110~2209.xlsx")

# Tool change & Spanker Cleaning & Mold cleaning

In [78]:
df4 = df3[df3['STATE_NAME_x'].isin(['Waiting for Response'])]

In [86]:
df4 = df3[df3['STATE_NAME_x'].isin(['Waiting For Technician'])]

In [53]:
df4 = df3[df3['STATE_NAME_y'].isin(['Change Collet'])]

In [44]:
df4 = df3[df3['STATE_NAME_y'].isin(['Regular Mold Cleaning'])]

In [77]:
df4 = df3[df3['STATE_NAME_x'].isin(['Undefined Waiting'])]

In [80]:
df4 = df3[df3['STATE_NAME_y'].isin(['Change Tool'])]

In [66]:
df4 = df3[df3['STATE_NAME_y'].isin(['Waiting (Tool change OP)'])]

In [44]:
df4 = df3[df3['STATE_NAME_y'].isin(['Change Device'])]

In [37]:
df4 = df3[df3['STATE_NAME_x'].isin(['Undefined Waiting','Machine Failure','Material Problem','IT Problem','Waiting For Technician'])]

In [54]:
df4 = df4[(df4.seconds > 180)] #Tool Change

In [55]:
df4 = df4[(df4.seconds < 5400)] # Tool Change

In [45]:
df4 = df4[(df4.seconds > 600)] # Mold Cleaning

In [46]:
df4 = df4[(df4.seconds < 7200)] #Mold Cleaning

In [38]:
df4 = df4[(df4.seconds > 3600)]

In [46]:
df4

,week,Process,EQ_ID,Lotid,Product type,STATE_NAME_x,STATE_NAME_y,seconds
3800681,-44,LAW,LAW004,2Z1460GG,IKCM10H60GA,Change Setup,Change Device,3110
3801348,-42,LAW,LAW004,2Z1470E4,IKCM10H60GA,Change Setup,Change Device,4241
3802234,-41,LAW,LAW004,2Z1490DJ,IKCM15L60GA,Change Setup,Change Device,3036
3807812,-30,LAW,LAW004,2Z2080B3,IKCM15L60GA,Change Setup,Change Device,1713
3808075,-29,LAW,LAW004,2Z2080QP,IKCM15L60GA,Change Setup,Change Device,1610
...,...,...,...,...,...,...,...,...
4599784,-14,LAW,LAW043,2Z22402S,IGCM10F60GA,Change Setup,Change Device,13913
4601019,-12,LAW,LAW043,2Z22605A,IKCM10H60GA,Change Setup,Change Device,3415
4601469,-10,LAW,LAW043,2Z2260L8,IKCM10L60GA,Change Setup,Change Device,9238
4602438,-9,LAW,LAW043,2Z229024,IKCM10H60GA,Change Setup,Change Device,2151


In [45]:
df4['seconds'].mean()/60

123.85406824146982

In [51]:
df4.shape[0]/13430

0.009456440804169769

In [59]:
df4 = df4[(df4.Month == '2023-01')]

In [45]:
df4.sort_values(by=['Month'], inplace=True)

C:\Users\parkpaul\AppData\Local\Temp\ipykernel_4824\2808672820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.sort_values(by=['Month'], inplace=True)


In [58]:
#df3.groupby(['Month','EQ_ID'])[['Process']].count().reset_index().groupby(['Month'])[['EQ_ID']].count()

In [59]:
#df4.groupby(['EQ_ID']).sum().shape[0]

In [49]:
a.to_excel("Grupby2.xlsx")

In [47]:
k = pd.crosstab([df4.STATE_NAME_y], df4.week,values=df4.seconds,aggfunc='count',rownames=['LV6'],colnames=['month'])

In [48]:
k.to_excel("count.xlsx")

In [49]:
k

month,-46,-45,-44,-43,-42,-41,-40,-39,-38,-37,...,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7
LV6,,,,,,,,,,,,,,,,,,,,,
Change Device,4,7,9,8,15,12,8,8,9,8,...,11,6,8,10,12,16,9,7,8,3


In [47]:
v = pd.crosstab([df4.STATE_NAME_y], df4.Month,values=df4.seconds,aggfunc=sum,rownames=['LV6'],colnames=['month'])

In [54]:
c.to_excel("time2.xlsx")

In [132]:
v/3600

month,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
LV6,,,,,,,,,
Change Collet,71.328056,83.293056,106.864167,94.908611,66.095556,115.627222,133.7775,92.686667,81.301944


In [53]:
c = v/k/60

In [48]:
v/k/60

month,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08
LV6,,,,,,,,,
Change Device,121.281481,164.466667,111.909167,192.823016,117.261111,93.170833,43.759259,101.910256,105.252778


In [43]:
a = []

In [41]:
UW = len(df3[df3['STATE_NAME_y'] == 'Waiting (Technician) PAT'])

In [44]:
a.append((UW)/(df3.groupby(['Lotid']).count().shape[0]))

In [45]:
a.append((df3[df3['STATE_NAME_y'] == 'Waiting (Technician) PAT']['seconds'].sum())/(UW))

In [46]:
a

[0.6282810428634555, 854.2266141510761]

In [ ]:
one_page_MMO(df2, pq)

In [42]:
def one_page_MMO(df2, pq):
    

    def append_df_to_excel(filename, k, sheet_name='Sheet1', startrow=None,
                           truncate_sheet=False,
                           **to_excel_kwargs):


        # ignore [engine] parameter if it was passed
        if 'engine' in to_excel_kwargs:
            to_excel_kwargs.pop('engine')

        writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a',if_sheet_exists='overlay')
        
        try:
            FileNotFoundError
        except NameError:
            FileNotFoundError = IOError
            
        try:
            # try to open an existing workbook
            writer.book = load_workbook(filename)

            # get the last row in the existing Excel sheet
            # if it was not specified explicitly
            if startrow is None and sheet_name in writer.book.sheetnames:
                startrow = writer.book[sheet_name].max_row

            # truncate sheet
            if truncate_sheet and sheet_name in writer.book.sheetnames:
                # index of [sheet_name] sheet
                idx = writer.book.sheetnames.index(sheet_name)
                # remove [sheet_name]
                writer.book.remove(writer.book.worksheets[idx])
                # create an empty sheet [sheet_name] using old index
                writer.book.create_sheet(sheet_name, idx)

            # copy existing sheets
            writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
        except FileNotFoundError:
            # file does not exist yet, we will create it
            pass
            
        if startrow is None:
            startrow = 0
        #with pd.ExcelWriter('recent_OEE3.xlsx', mode='a') as writer:
        k.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
        
        writer.save()
        
    filename = r'C:\Users\ParkPaul\Documents\Python\UDW_UW_T.xlsx'

    for index, row in pq.iterrows():
        k = row
        lst = row['EQ_ID'].split(',')
        lst = [x.strip(' ') for x in lst]
        df3 = df2[df2['EQ_ID'].isin(lst)]
        
        a=[]
        try:
            UW = len(df3[df3['STATE_NAME_x'] == 'Undefined Waiting'])
            T = len(df3[df3['STATE_NAME_x'] == 'Waiting For Technician'])
            a.append((UW)/(df3.groupby(['Lotid']).count().shape[0]))
            a.append((T)/(df3.groupby(['Lotid']).count().shape[0]))
            a.append((df3[df3['STATE_NAME_x'] == 'Undefined Waiting']['seconds'].sum())/(UW))
            a.append((df3[df3['STATE_NAME_x'] == 'Waiting For Technician']['seconds'].sum())/(T))
            
        except ZeroDivisionError:
            pass
        

        a = pd.DataFrame(a)
        a = a.transpose()
        
        a = a.rename(columns={0: row['Device'] + "_" + row['Process']})


        append_df_to_excel(filename, a)